### Imports

In [2]:
import os
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import matthews_corrcoef
from sklearn.preprocessing import LabelEncoder
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import KFold

### Functions

In [3]:
#Accuracy
def _accuracy(actual, predicted):
    return accuracy_score(actual, predicted)

#Sensitivity(Recall)
def _sens(actual, predicted):
    return precision_recall_fscore_support(actual, predicted,average='weighted')[1]

#MCC
def _mcc(actual, predicted):
    return matthews_corrcoef(actual,predicted)

#Sub-Smapling Data
def _downSample(_dataset, _size):

    # Creating the unique class names and then sorting
    _classes=sorted(_dataset['label'].unique())
    # Putting each class in a differnet data frame
    _data_by_class=[_dataset[_dataset['label']==_class] for _class in _classes ]
    
    _folds_by_class_=[]
    _num_of_folds=5
    # Creating folds from each class
    for _item in _data_by_class:
        np.random.seed(2)
        _item=_item.reindex(np.random.permutation(_item.index))
        co=_size//_num_of_folds
        _folds_by_class_.append([_item[(i*co):((i*co)+co)] for i in range(_num_of_folds)])
    
    # Creating Folds from the whole feature
    # by concatenation of each fold from each class
    _folds_=[
        pd.concat([_folds_by_class_[i][j] for i in range(len(_folds_by_class_))]) 
        for j in range(_num_of_folds)
        ]
    
    # Train,Test out of 5 folds
    _train_test_=[]
    for ii in range(_num_of_folds):
        _test__ =_folds_[ii]
        _train__=pd.concat([_folds_[xx] for xx in range(_num_of_folds) if xx!=ii])
        _train_test_.append([_train__,_test__])
    
    return _train_test_

#Splits Data into [X_train, X_test, y_train, y_test]
def _split_(_train,_test):
    
    y_test = _test['label'].values
    del _test['label']
    X_test = _test.values
    
    y_train = _train['label'].values
    del _train['label']
    X_train = _train.values

    
    return [X_train, X_test, y_train, y_test]

#Prediciton
def _predict(_train_test,_g,_c):
    
    X_train, X_test, y_train, y_test = _train_test
    
    _gamma,_C =_g,_c
    #classifier = OneVsRestClassifier( SVC(kernel='rbf', gamma=_gamma,C=_C) )
    classifier=SVC(kernel='rbf', gamma=_gamma,C=_C, decision_function_shape='ovr')    

    #Training the algorithm on training data
    classifier.fit(X_train, y_train)

    # Predicting the Test set results
    y_pred = classifier.predict(X_test)

    return [y_test.copy(),y_pred.copy()]

### Main Program

In [4]:
kf = KFold(n_splits=5)

In [7]:
data8=pd.read_csv(os.path.join('dataset','aac8.csv'))

In [8]:
y = data8['label'].values
le=LabelEncoder()
y=le.fit_transform(y)
data8['label']=y

### Results

#### No Shuffling

In [31]:
kf = KFold(n_splits=5)
res=[]
for train, test in kf.split(data8.copy()):

    df_train=pd.DataFrame(data8.iloc[train])
    df_test=pd.DataFrame(data8.iloc[test])
    y8_test,y8_pred=_predict(_split_(df_train.copy(),df_test.copy()).copy(),0.02,4.1)
    
    res.append([_accuracy(y8_test.copy(),y8_pred.copy()),
                  _mcc(y8_test.copy(),y8_pred.copy()),
                  _sens(y8_test.copy(),y8_pred.copy())
              ])

# print(aacuracy, MCC, Sensiticity)
print('_acc = ',sum([_num[0] for _num in res])/5)
print('_mcc = ',sum([_num[1] for _num in res])/5)
print('_sens = ',sum([_num[2] for _num in res])/5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score 

_acc =  0.39057971014492754
_mcc =  0.05675550557214313
_sens =  0.39057971014492754


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/classification.py:543: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


#### On Shuffled Dataset

In [33]:
np.random.seed(2)
data8s=data8.copy()
data8s=data8s.reindex(np.random.permutation(data8s.index))

kf = KFold(n_splits=5)
_res=[]
for train, test in kf.split(data8s.copy()):
    
    df_train=pd.DataFrame(data8s.iloc[train])
    df_test=pd.DataFrame(data8s.iloc[test])
    y8_test,y8_pred=_predict(_split_(df_train.copy(),df_test.copy()).copy(),0.02,4.1)
    
    _res.append([_accuracy(y8_test.copy(),y8_pred.copy()),
                  _mcc(y8_test.copy(),y8_pred.copy()),
                  _sens(y8_test.copy(),y8_pred.copy())
              ])

# print(aacuracy, MCC, Sensiticity)
print('_acc = ',sum([_num[0] for _num in _res])/5)
print('_mcc = ',sum([_num[1] for _num in _res])/5)
print('_sens = ',sum([_num[2] for _num in _res])/5)

_acc =  0.5376811594202899
_mcc =  0.34319881640471295
_sens =  0.5376811594202899


#### On Down-Sampled Dataset

In [36]:
_train_test__ = _downSample(data8.copy(),60)
_res__=[]
for train, test in _train_test__:

    y8_test__,y8_pred__=_predict(_split_(train.copy(),test.copy()),0.02,4.1)
    
    _res__.append([_accuracy(y8_test__.copy(),y8_pred__.copy()),
          _mcc(y8_test__.copy(),y8_pred__.copy()),
          _sens(y8_test__.copy(),y8_pred__.copy())
              ])

print('_acc = ',sum([_num[0] for _num in _res__])/5)
print('_mcc = ',sum([_num[1] for _num in _res__])/5)
print('_sens = ',sum([_num[2] for _num in _res__])/5)

_acc =  0.38541666666666663
_mcc =  0.2994410825953192
_sens =  0.38541666666666663
